In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from bertopic import BERTopic

/home/cfoye/.pyenv/versions/impresso/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
acm_competitions = pd.read_feather("Data/ACM/AcmEPFL_paired.feather")

In [3]:
acm_competitions

,orig_index,Nom de l'objet,Programme,Adresse 1,Adresse 2,Adresse 3,Numéro postal,Localité,Canton - Département,Pays,...,Concours ouvert à,Notes de l'objet,Auteurs,Rôle de l'auteur,Dossiers reliés,Pièces reliées,CombinedArtists,is_jury,paired_competitons,comp_id
0,0,Concours pour le nouveau bâtiment des écoles d...,None,None,None,None,NaN,Vevey,None,Vaud,...,A. national,Ouvert par la Municipalité sur décret du 15 ma...,None,None,None,None,None,False,[1],0
1,1,Concours pour le nouveau bâtiment des écoles d...,None,None,None,None,NaN,Vevey,None,Vaud,...,None,Le concours est mentionné en 1878 (Eisenbahn v...,"Stadler +;Ritscher von Steinbeck +;Recordon, B...","3e prix;3e prix;Juré, rapporteur;Concurrent;2e...",None,None,"[[Stadler, Ritscher von Steinbeck, Recordon, B...",True,[0],1
2,2,Concours (1er degré) pour le monument du génér...,None,None,None,None,NaN,Genève,None,Suisse,...,National,Chercher dans Eisenbahn vol. V une annonce d'o...,"Thomas, Jules;Cain, Auguste-Nicolas;Rothpletz,...",Juré;Juré;Juré;Juré;Juré;Juré;Juré;Juré;Juré;J...,None,None,"[[Thomas, Jules], [Cain, Auguste-Nicolas], [Ro...",False,[1540],2
3,3,Concours restreint pour le nouveau Musée de la...,None,None,None,None,NaN,Mulhouse,None,None,...,None,"""Par suite d'un concours entre un nombre restr...","Rütti, Ludwig Friedrich von",Concurrent,None,None,"[[Rütti, Ludwig Friedrich von]]",True,[],3
4,4,Concours d'éléments analytiques : 2e classe : ...,None,None,None,None,NaN,None,None,None,...,None,None,None,None,None,0138.04.0001/262,None,False,[],4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381,2381,"Concours ""Reflet"" hôtel",None,None,None,None,NaN,Ouchy,Vaud,Suisse,...,None,None,None,None,0133.04.0316,None,None,False,[],2381
2382,2382,"SIA, commission des concours-641",None,None,None,None,NaN,None,None,None,...,None,None,None,None,0011.03.0037 ; 0011.03.0032 ; 0011.03.0034,None,None,False,[],2382
2383,2383,"SIA, commission des concours-642",None,None,None,None,NaN,None,None,None,...,None,None,None,None,0002.03.0102 ; 0002.03.0094 ; 0002.03.0101,None,None,False,[],2383
2384,2384,"Temple de Saint-Marc (Lausanne), concours",None,"Renens, chemin de, Sévery, avenue de",None,None,NaN,Lausanne,Vaud,Suisse,...,None,"Jury:Jean Tschumy, Edmond Virieux, arch. cant....",None,None,0003.04.0059,None,None,False,[],2384


In [4]:
nlp = spacy.load("fr_dep_news_trf", exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

In [5]:
from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(embedding_model=nlp.pipe, representation_model=representation_model)

In [6]:
acm_competitions = acm_competitions[
    (~acm_competitions['is_jury']) | 
    (acm_competitions['is_jury'] & acm_competitions['paired_competitons'].apply(lambda x: len(x) == 0 if isinstance(x, list) else False))
]


In [7]:
acm_competitions = acm_competitions.dropna(subset=['Date de début de l\'objet'])

In [8]:
acm_competitions

,orig_index,Nom de l'objet,Programme,Adresse 1,Adresse 2,Adresse 3,Numéro postal,Localité,Canton - Département,Pays,...,Concours ouvert à,Notes de l'objet,Auteurs,Rôle de l'auteur,Dossiers reliés,Pièces reliées,CombinedArtists,is_jury,paired_competitons,comp_id
0,0,Concours pour le nouveau bâtiment des écoles d...,None,None,None,None,NaN,Vevey,None,Vaud,...,A. national,Ouvert par la Municipalité sur décret du 15 ma...,None,None,None,None,None,False,[1],0
2,2,Concours (1er degré) pour le monument du génér...,None,None,None,None,NaN,Genève,None,Suisse,...,National,Chercher dans Eisenbahn vol. V une annonce d'o...,"Thomas, Jules;Cain, Auguste-Nicolas;Rothpletz,...",Juré;Juré;Juré;Juré;Juré;Juré;Juré;Juré;Juré;J...,None,None,"[[Thomas, Jules], [Cain, Auguste-Nicolas], [Ro...",False,[1540],2
4,4,Concours d'éléments analytiques : 2e classe : ...,None,None,None,None,NaN,None,None,None,...,None,None,None,None,None,0138.04.0001/262,None,False,[],4
5,5,Concours d'émulation - 2e classe - Esquisse: U...,None,None,None,None,NaN,Paris,None,France,...,None,"""Dans les grands palais, la chambre coucher d...","Lesueur, Jean-Baptiste-Cicéron",None,None,0138.04.0001/71,"[[Lesueur, Jean-Baptiste-Cicéron]]",False,[],5
7,7,Concours d'éléments analytiques : 2e classe : ...,None,None,None,None,NaN,None,None,None,...,None,None,None,None,None,0138.04.0001/204 ; 0138.04.0001/268,None,False,[],7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,2145,Concours pour l'agrandissement de la Faculté d...,None,None,None,None,NaN,Fribourg,None,Suisse,...,A. cantonal,Ouvert par la Fondation pour le développement ...,None,None,None,None,None,False,[2076],2145
2146,2146,concours pour le restaurant des Iles: projet L...,Hôtellerie et tourisme : restaurant,None,None,None,1950.0,Sion,None,Suisse,...,None,concours ouvert aux architectes ayant leur dom...,"Morisod, Paul;Bourgeoisie de Sion;Furrer, Edou...",Architecte;Maître de l'ouvrage;Architecte,0076.04.0138 ; 0076.02.0064 ; 0076.03.0022,None,"[[Morisod, Paul], [Bourgeoisie de Sion], [Furr...",False,[],2146
2147,2147,Concours d'ornement et d'ajustement - 1ère cla...,Monument,None,None,None,NaN,None,None,None,...,None,None,None,None,None,0138.04.0001/182 ; 0138.04.0001/181,None,False,[],2147
2157,2157,Concours d'idées pour la construction d'un Pal...,Culture et vie sociale,None,None,None,NaN,Genève,None,Suisse,...,None,"Projet nommé Centric. Bâtiment de plan carré, ...","Gaillard, André;Gaillard, Francis",Architecte;Architecte,0143.04.0360,None,"[[Gaillard, André], [Gaillard, Francis]]",False,[],2157


In [9]:
docs = acm_competitions['Nom de l\'objet'].dropna().to_list()

In [10]:
from spacy_cleaner import processing, Cleaner
cleaner = Cleaner( 
    nlp,
    processing.remove_stopword_token,
    # processing.replace_punctuation_token,
    # processing.mutate_lemma_token,
)
texts = cleaner.clean(docs)

Cleaning Progress: 100%|██████████| 1331/1331 [00:30<00:00, 44.13it/s]


In [11]:
texts

['Concours bâtiment écoles filles Vevey',
 'Concours ( 1er degré ) monument général Dufour',
 'Concours éléments analytiques : 2e classe : Rendu : façade phare',
 'Concours émulation - 2e classe - Esquisse : chambre coucher princière',
 'Concours éléments analytiques : 2e classe : Rendu : détails portique ordre dorique romain',
 'Concours émulation - 2e classe - Esquisse : bureau recrutement chef - lieu département',
 'Concours école horlogerie Locle',
 'Concours restreint bâtiment salles conférences Neuchâtel',
 'Concours : Prix Godeboeuf : 1885 : horloge adossée',
 'Concours émulation : 1ère classe : Esquisse : entrée principale grand cimetière',
 "Concours émulation : 2e classe : Esquisse : banc d'&#339;uvre Eglise paroissiale petite ville",
 'Concours émulation : 2e classe : Rendu : théâtre ville second ordre',
 'Concours fontaine monumentale élever Place Neuve',
 'Concours : Prix Godeboeuf : 1886 : lampadaire électrique',
 'Concours émulation : 1ère classe : Esquisse : monument co

In [12]:
texts = [text.replace("concours ", "").strip() for text in texts]
texts = [text.replace("Concours ", "").strip() for text in texts]
texts

['bâtiment écoles filles Vevey',
 '( 1er degré ) monument général Dufour',
 'éléments analytiques : 2e classe : Rendu : façade phare',
 'émulation - 2e classe - Esquisse : chambre coucher princière',
 'éléments analytiques : 2e classe : Rendu : détails portique ordre dorique romain',
 'émulation - 2e classe - Esquisse : bureau recrutement chef - lieu département',
 'école horlogerie Locle',
 'restreint bâtiment salles conférences Neuchâtel',
 ': Prix Godeboeuf : 1885 : horloge adossée',
 'émulation : 1ère classe : Esquisse : entrée principale grand cimetière',
 "émulation : 2e classe : Esquisse : banc d'&#339;uvre Eglise paroissiale petite ville",
 'émulation : 2e classe : Rendu : théâtre ville second ordre',
 'fontaine monumentale élever Place Neuve',
 ': Prix Godeboeuf : 1886 : lampadaire électrique',
 'émulation : 1ère classe : Esquisse : monument commémoratif Exposition universelle',
 'émulation : 1ère classe : Rendu : odéon Salle concert',
 'musée beaux - Arts',
 ': Prix Godeboeuf

In [13]:
topics, probs = topic_model.fit_transform(texts)

/home/cfoye/.pyenv/versions/impresso/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [14]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,225,-1_société_meilleures_constructions_honneur,"[société, meilleures, constructions, honneur, ...",[constructions Société suisse assurances génér...
1,0,149,0_école_ecole_écoles_institut,"[école, ecole, écoles, institut, université, u...","[école secondaire Porrentruy, construction éco..."
2,1,137,1_lausanne_zürich_palais_institut,"[lausanne, zürich, palais, institut, ecole, éc...","[école supérieure jeunes filles Lausanne, Pala..."
3,2,106,2_aménagement_villamont_touristique_quais,"[aménagement, villamont, touristique, quais, a...","[aménagement quartier Villamont place Faucon, ..."
4,3,61,3_bâtiment_bâtiments_bâtimenbts_coudre,"[bâtiment, bâtiments, bâtimenbts, coudre, scol...",[appel avant-projets relatifs bâtimenbts place...
5,4,55,4_maison_maisons_congrès_villas,"[maison, maisons, congrès, villas, européenne,...","[maison Congrès , Genève, Maison congrès , [ G..."
6,5,53,5_centre_scolaire_viège_tunis,"[centre, scolaire, viège, tunis, bresse, saagi...",[centre scolaire ( cycle orientation ) Collomb...
7,6,43,6_autoroute_lausanne_pont_ponts,"[autoroute, lausanne, pont, ponts, autoroutier...",[autoroute Lausanne-Saint-Maurice N9 Pont Veve...
8,7,40,7_ville_extension_lille_morat,"[ville, extension, lille, morat, suburbaines, ...","[plan extension ville Bienne, plan extension v..."
9,8,39,8_musée_muséum_museum_histoire,"[musée, muséum, museum, histoire, sculpture, a...","[Musée Beaux - Arts , Bâle, décoration façade ..."


In [15]:
doc_topic_df = topic_model.get_document_info(texts)


In [26]:
fig = topic_model.visualize_topics()
fig.show()

In [17]:
acm_competitions["Localité"].value_counts()

Localité
Lausanne                156
Genève                  115
Neuchâtel                37
Bern                     34
Fribourg                 33
                       ... 
Thusis                    1
Klosters                  1
Villars-Sainte-Croix      1
Chexbres                  1
Vollèges                  1
Name: count, Length: 357, dtype: int64

In [18]:
acm_competitions

,orig_index,Nom de l'objet,Programme,Adresse 1,Adresse 2,Adresse 3,Numéro postal,Localité,Canton - Département,Pays,...,Concours ouvert à,Notes de l'objet,Auteurs,Rôle de l'auteur,Dossiers reliés,Pièces reliées,CombinedArtists,is_jury,paired_competitons,comp_id
0,0,Concours pour le nouveau bâtiment des écoles d...,None,None,None,None,NaN,Vevey,None,Vaud,...,A. national,Ouvert par la Municipalité sur décret du 15 ma...,None,None,None,None,None,False,[1],0
2,2,Concours (1er degré) pour le monument du génér...,None,None,None,None,NaN,Genève,None,Suisse,...,National,Chercher dans Eisenbahn vol. V une annonce d'o...,"Thomas, Jules;Cain, Auguste-Nicolas;Rothpletz,...",Juré;Juré;Juré;Juré;Juré;Juré;Juré;Juré;Juré;J...,None,None,"[[Thomas, Jules], [Cain, Auguste-Nicolas], [Ro...",False,[1540],2
4,4,Concours d'éléments analytiques : 2e classe : ...,None,None,None,None,NaN,None,None,None,...,None,None,None,None,None,0138.04.0001/262,None,False,[],4
5,5,Concours d'émulation - 2e classe - Esquisse: U...,None,None,None,None,NaN,Paris,None,France,...,None,"""Dans les grands palais, la chambre coucher d...","Lesueur, Jean-Baptiste-Cicéron",None,None,0138.04.0001/71,"[[Lesueur, Jean-Baptiste-Cicéron]]",False,[],5
7,7,Concours d'éléments analytiques : 2e classe : ...,None,None,None,None,NaN,None,None,None,...,None,None,None,None,None,0138.04.0001/204 ; 0138.04.0001/268,None,False,[],7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,2145,Concours pour l'agrandissement de la Faculté d...,None,None,None,None,NaN,Fribourg,None,Suisse,...,A. cantonal,Ouvert par la Fondation pour le développement ...,None,None,None,None,None,False,[2076],2145
2146,2146,concours pour le restaurant des Iles: projet L...,Hôtellerie et tourisme : restaurant,None,None,None,1950.0,Sion,None,Suisse,...,None,concours ouvert aux architectes ayant leur dom...,"Morisod, Paul;Bourgeoisie de Sion;Furrer, Edou...",Architecte;Maître de l'ouvrage;Architecte,0076.04.0138 ; 0076.02.0064 ; 0076.03.0022,None,"[[Morisod, Paul], [Bourgeoisie de Sion], [Furr...",False,[],2146
2147,2147,Concours d'ornement et d'ajustement - 1ère cla...,Monument,None,None,None,NaN,None,None,None,...,None,None,None,None,None,0138.04.0001/182 ; 0138.04.0001/181,None,False,[],2147
2157,2157,Concours d'idées pour la construction d'un Pal...,Culture et vie sociale,None,None,None,NaN,Genève,None,Suisse,...,None,"Projet nommé Centric. Bâtiment de plan carré, ...","Gaillard, André;Gaillard, Francis",Architecte;Architecte,0143.04.0360,None,"[[Gaillard, André], [Gaillard, Francis]]",False,[],2157


In [19]:
acm_competitions['Date de début de l\'objet'].dt.year

0       1872
2       1877
4       1882
5       1883
7       1884
        ... 
2145    1959
2146    1978
2147    1887
2157    1969
2158    1981
Name: Date de début de l'objet, Length: 1331, dtype: int32

In [20]:
topics_over_time = topic_model.topics_over_time(texts, acm_competitions['Date de début de l\'objet'].dt.year, nr_bins=20)

In [27]:
topics_over_time

,Topic,Words,Frequency,Timestamp,Name
0,0,"ecole, école, écoles, institut, université",1,1871.868,0_école_ecole_écoles_institut
1,1,"lausanne, vieillesse, palais, hospice, tribunal",2,1871.868,1_lausanne_zürich_palais_institut
2,19,"monument, dufour, de, degré, peintre",1,1871.868,19_monument_rambert_fondation_clarens
3,-1,"coulouvrenière, monumentale, fontaine, godeboe...",3,1878.600,-1_société_meilleures_constructions_honn...
4,0,"école, ecole, écoles, horlogerie, institut",1,1878.600,0_école_ecole_écoles_institut
...,...,...,...,...,...
340,5,"routes, nationales, tour, orientation, centre",2,1997.400,5_centre_scolaire_viège_tunis
341,6,"rhône, pont, tunnel, montreux, souste",2,1997.400,6_autoroute_lausanne_pont_ponts
342,18,"cifom, école, technique, locle, salle",1,1997.400,18_gymnastique_salle_salles_école
343,20,"gymnase, payerne, broye, montagnes, intercantonal",1,1997.400,20_cantonal_cantonale_gymnase_canton


In [23]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=5)
